In [1]:
import pandas as pd
import sqlite3

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
con = sqlite3.Connection("./switrs.sqlite")

<strong style="color:red;font-size:2rem;">WARNING: This notebook will use a lot of RAM!</strong>
    
 This notebook will use about 8-10 gigs of RAM loading the two tables into dataframes.
 
 If you want to use less, make this number below larger:

In [4]:
REDUCTION_FACTOR = 10

In [5]:
collisions = pd.read_sql(
    f"SELECT * FROM collisions WHERE case_id % {REDUCTION_FACTOR} = 0", 
    con, 
    parse_dates=["collision_date"]
)

In [6]:
# Select parties with the same case_id as selected for collisions
parties = pd.read_sql(f"SELECT * FROM parties WHERE case_id % {REDUCTION_FACTOR} = 0", con)

# Data Science Interview Practice: Data Manipulation

I often get asked by newly-minted PhDs trying to get their first data job:

> How can I prepare for dataset-based interviews? Do you have any examples of
> datasets to practice with?

I never had a good answer. I would tell them about how the interviews worked,
but I wished I had something to share that they could get their hands on.

As of today, that's changing. In this post I put together a series of practice
questions like the kind you might see (or be expected to come up with) in a
hands-on data interview using the [curated and hosted dataset of California
Traffic accidents][switrs_dataset]. The dataset is available for download from
both [Kaggle][kaggle] and [Zenodo][zenodo], and I even have an [example
notebook][example_notebook] demonstrating how to work with the data entirely
online within Kaggle.

[switrs_dataset]: https://alexgude.com/blog/switrs-sqlite-hosted-dataset/

[kaggle]: https://www.kaggle.com/alexgude/california-traffic-collision-data-from-switrs

[zenodo]: https://zenodo.org/record/4284843

[example_notebook]: https://www.kaggle.com/alexgude/starter-california-traffic-collisions-from-switrs

## Interview Format

As I mentioned in [my post about my most recent interview
experience][last_post], data science and machine learning interviews have
become more practical, covering tasks that show up in the day-to-day work of a
data scientist instead of hard but irrelevant problems. One common interview
type involves working with a dataset, answering some simple questions about
it, and then building some simple features.

[last_post]: https://alexgude.com/blog/interviewing-for-data-science-positions-in-2020/

Generally these interviews use Python and [Pandas][pandas] or pure SQL.
Sometimes the interviewer has a set of questions for you to answer and
sometimes they want you to come up with your own.

[pandas]: https://en.wikipedia.org/wiki/Pandas_(software)

To help people prepare, I have created a set of questions similar to what you
would get in a real interview. For the exercise you will be using the SWITRS
dataset. I have included a notebook to get you started in Pandas or SQL. The
solution notebooks can be found at the very end.

Good luck, and if you have any questions or suggestions please reach out to me
on Twitter: [@alex_gude][twitter]

[twitter]: https://twitter.com/alex_gude

## Questions

### How many collisions are there in the dataset?

A good first thing to check is "How much data am I dealing with?"

Each row in the collisions database represents one collision, so the solution
is nice and short:

In [7]:
len(collisions)

899955

### What percent of collisions involve males aged 16--25?

Young men are famously unsafe drivers so let's look at how many collisions
they're involved in.

In [8]:
criteria = (
    (parties["party_sex"] == "male")
    & (parties["party_age"].between(16, 25))
)

# There are a lot of NULLs, which I assume are uncorrelated to 
# the real Age and Sex, so I remove them.
denominator_criteria = (
    (~parties["party_sex"].isna())
    & (~parties["party_age"].isna())
)

len(parties[criteria]["case_id"].unique()) / len(parties[denominator_criteria]["case_id"].unique())

0.25815681787972583

This won't match the answer in the post because we are randomly sampling.

### How many solo motorcycle crashes are there per year?

A _"solo"_ crash is one where the driver runs off the road or hits a
stationary object. How many solo motorcycle crashes were there each year? Why
does 2020 seem to (relatively) have so few?

In [9]:
criteria = (
    (collisions["party_count"] == 1)  # Solo crash
    & (collisions["motorcycle_collision"] == True)  # Is motorcycle collision
)

# Select a single column to speed computation, it could be any column, but I used jurisdiction.
collisions[criteria]["jurisdiction"].groupby(collisions["collision_date"].dt.year).count()

collision_date
2001    143
2002    336
2003    385
2004    424
2005    405
2006    418
2007    447
2008    529
2009    455
2010    408
2011    403
2012    419
2013    434
2014    393
2015    438
2016    455
2017    445
2018    421
2019    382
2020    350
Name: jurisdiction, dtype: int64

The count is low in 2020 primarily because the data doesn't cover the whole
year. It is also low due to the COVID pandemic keeping people off the streets,
at least initially. To differentiate these two causes we could compare month
by month to last year.

### What make of vehicle has the largest fraction of accidents on the weekend? During the work week?

Weekdays are generally commute and work-related traffic, while weekends
involves recreational travel. Do we see different vehicles involved in
collisions on these days?

Only consider vehicle makes with at least 1,000 collisions, in order to focus
only on common vehicles where the difference between weekend and weekday usage
will be significant.

This query is tricky. We need to aggregate collisions by vehicle make, which
means we need the parties table. We also care about when the crash happened,
which means we need the collisions table. So we need to join these two tables
together.

In an interview setting, I would write two simpler queries: one
that gets the highest weekend fraction and one that gets the highest weekday
fraction with a lot of copy and pasted code. This is a lot easier to work out.
Here is an example of one of those queries:

In [10]:
collisions["day_name"] = collisions["collision_date"].dt.day_name()
collisions["is_weekend"] = collisions["day_name"].isin(("Saturday", "Sunday"))
collisions["is_weekday"] = ~collisions["is_weekend"]

merged = pd.merge(
    parties[["case_id", "vehicle_make"]], 
    collisions[["case_id", "is_weekend", "is_weekday"]], 
    how="inner",
    on=["case_id"],
)

grouped = merged[["vehicle_make", "is_weekend", "is_weekday"]].groupby("vehicle_make").sum()
grouped["total"] = grouped["is_weekend"] + grouped["is_weekday"]
selected = grouped[grouped["total"] >= 1_000].copy()

selected["weekend_ratio"] = selected["is_weekend"] / selected["total"]
selected["weekday_ratio"] = selected["is_weekday"] / selected["total"]

top_weekend = selected.sort_values(["weekend_ratio"], ascending=False).head(1)
top_weekday = selected.sort_values(["weekday_ratio"], ascending=False).head(1)

answer = pd.concat([top_weekend, top_weekday])
answer.head()

,is_weekend,is_weekday,total,weekend_ratio,weekday_ratio
vehicle_make,,,,,
HARLEY-DAVIDSON,1849.0,2972.0,4821.0,0.383530,0.616470
PTRB,119.0,1151.0,1270.0,0.093701,0.906299


### How many different values represent "Toyota" in the Parties database? How would you go about correcting for this?

Data is **_never_** as clean as you would hope,  and this applies even to the
[curated SWITRS dataset][switrs_dataset]. How many different ways does
"Toyota" show up?

[switrs_dataset]: https://alexgude.com/blog/switrs-sqlite-hosted-dataset/

What steps would you take to fix this problem?


This is a case where there is no _right_ answer. You can get a more and more
correct answer as you spend more time, but at some point you have to decide it
is good enough.

In [11]:
regex = r"(TOYOTA.*|TOY.*|TY.*)"

just_makes = parties[["vehicle_make", "party_sex"]]\
               .groupby("vehicle_make")\
               .count()\
               .reset_index()\
               .rename(columns={"party_sex": "count"})\
               .sort_values("count", ascending=False)

just_makes[just_makes["vehicle_make"].str.match(regex)]

,vehicle_make,count
1587,TOYOTA,210832
1581,TOYO,15912
1600,TOYT,14212
1586,TOYOT,264
1577,TOY,232
1604,TOYTA,32
1588,TOYOTA/,23
1606,TOYTOA,7
1583,TOYOA,6
1605,TOYTO,5


Most of those look like they mean Toyota, although Tymco is a different
company that makes street sweepers.

Here is how I would handle this issue: the top 5 make up the vast majority of
entries. I would fix those by hand and move on. More generally it seems that
makes are represented mostly by their name or a four-letter abbreviation. It
wouldn't be too hard to detect and fix these for the most common makes.